Sharer / Coder / Listener:

Lekshmi (les505@g.harvard.edu)

Ninon Becquart (nbecquart@hsph.harvard.edu)

Gerald Arocena (geraldarocena@g.harvard.edu)

# Lecture 22
## Thursday, November 19th 2020
### Joins with `SQLite`, `pandas`

## Starting Up
You can connect to the saved database from last time if you want.  Alternatively, for extra practice, you can just recreate it from the datasets provided in the `.txt` files.  That's what I'll do.

### Comments
You should make a copy of this notebook and call it `Lecture22_Exercises.ipynb`. Do all excercises in a single cell immediately after the Exercise statement similarly to what you did in Lecture 21.

In [35]:
import sqlite3
import numpy as np
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

db = sqlite3.connect('L22DB.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_init TEXT, 
               party TEXT NOT NULL)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

db.commit()

with open ("candidates.txt") as candidates:
    next(candidates) # jump over the header
    for line in candidates.readlines():
        cid, first_name, last_name, middle_name, party = line.strip().split('|')
        vals_to_insert = (int(cid), first_name, last_name, middle_name, party)
        cursor.execute('''INSERT INTO candidates 
                  (id, first_name, last_name, middle_init, party)
                  VALUES (?, ?, ?, ?, ?)''', vals_to_insert)

with open ("contributors.txt") as contributors:
    next(contributors)
    for line in contributors.readlines():
        cid, last_name, first_name, middle_name, street_1, street_2, \
            city, state, zip_code, amount, date, candidate_id = line.strip().split('|')
        vals_to_insert = (last_name, first_name, middle_name, street_1, street_2, 
                          city, state, int(zip_code), amount, date, candidate_id)
        cursor.execute('''INSERT INTO contributors (last_name, first_name, middle_name, 
                           street_1, street_2, city, state, zip, amount, date, candidate_id) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)

candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]

def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [col[i] for col in q]
    return pd.DataFrame.from_dict(framelist)

from IPython.display import display

## Recap
Last time, you played with a bunch of `SQLite` commands to query and update the tables in the database.

One thing we didn't get to was how to query the contributors table based off of a query in the candidates table.  For example, suppose you want to query which contributors donated to Obama. You could use a nested `SELECT` statement to accomplish that.

In [36]:
query = '''SELECT * FROM contributors WHERE candidate_id = (SELECT id from candidates WHERE last_name = "Obama")'''
display(viz_tables(contributor_cols, query))

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,26,Buckler,Steve,,24351 Armada Dr,,Dana Point,CA,926291306,50.00,2007-07-30,20
1,27,Buckler,Steve,,24351 Armada Dr,,Dana Point,CA,926291306,25.00,2007-08-16,20
2,28,Buckheit,Bruce,,8904 KAREN DR,,FAIRFAX,VA,220312731,100.00,2007-09-19,20
3,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.00,2007-08-14,20
4,30,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,-2300.00,2007-08-14,20
5,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.00,2007-08-14,20
6,32,Buck,Thomas,,4206 Terrace Street,,Kansas City,MO,64111,100.00,2007-09-25,20
7,33,Buck,Jay,K.,1855 Old Willow Rd Unit 322,,Northfield,IL,600932918,200.00,2007-09-12,20
8,34,Buck,Blaine,M,45 Eaton Ave,,Camden,ME,48431752,2300.00,2007-09-30,20
9,35,Buck,Barbara,,1780 NE 138th St,,North Miami,FL,331811316,50.00,2007-09-13,20


# Joins
The last example involved querying data from multiple tables.

In particular, we combined columns from the two related tables (related through the `FOREIGN KEY`).

This leads to the idea of *joining* multiple tables together.  `SQL` has a set of commands to handle different types of joins.  `SQLite` does not support the full suite of join commands offered by `SQL` but you should still be able to get the main ideas from the limited command set.

We'll begin with the `INNER JOIN`.

## INNER JOIN
The idea here is that you will combine the tables if the values of certain columns are the same between the two tables.  In our example, we will join the two tables based on the candidate id.  The result of the `INNER JOIN` will be a new table consisting of the columns we requested and containing the common data.  Since we are joining based off of the candidate id, we will not be excluding any rows.

### Example

Here are two tables.  Table A has the form:

| nA   | attr  | idA |
| :::: | ::::: | ::: |
| s1   | 23    | 0   |
| s2   | 7     | 2   |

and table B has the form:

| nB   | attr  | idB |
| :::: | ::::: | ::: |
| t1   | 60    | 0   |
| t2   | 14    | 7   |
| t3   | 22    | 2   |

Table A is associated with Table B through a foreign key on the id column.

If we join the two tables by comparing the id columns and selecting the nA, nB, and attr columns then we'll get 

| nA   | A.attr  | nB  | B.attr |
| :::: | ::::::: | ::: | :::::: |
| s1   | 23      | t1  | 60     |
| s2   | 7       | t3  | 22     |


The `SQLite` code to do this join would be 

```SQL
SELECT nA, A.attr, nB, B.attr FROM A INNER JOIN B ON B.idB = A.idA
```

Notice that the second row in table B is gone because the id values are not the same.

### Thoughts
What is `SQL` doing with this operation?  It may help to visualize this with a Venn diagram.  Table A has rows with values corresponding to the `idA` attribute.  Column B has rows with values corresponding to the `idB` attribute.  The `INNER JOIN` will combine the two tables such that rows with common entries in the `id` attributes are included.  We essentially have the following Venn diagram.

![](../fig/inner_join.png)

## Exercises
1. Using an `INNER JOIN`, join the candidates and contributors tables by comparing the `candidate_id` and `candidates_id` columns.  Display your joined table with the columns `contributors.last_name`, `contributors.first_name`, and `candidates.last_name`.
2. Do the same inner join as in the last part, but this time append a `WHERE` clause to select a specific candidate's last name.

In [37]:
# Using an INNER JOIN, join the candidates and contributors tables by comparing the candidate_id and candidates_id columns. Display your joined table with the columns contributors.last_name, contributors.first_name, and candidates.last_name.
print('Inner join - part 1')
query = '''SELECT
  contributors.last_name,
  contributors.first_name,
  candidates.last_name
FROM
  candidates
INNER JOIN
  contributors
  ON contributors.candidate_id = candidates.id'''

display(viz_tables(['last_name', 'first_name', 'last_name'], query))

print('Inner join - part 2')
query_2 = '''SELECT
  contributors.last_name,
  contributors.first_name,
  candidates.last_name
FROM
  candidates
INNER JOIN
  contributors
  ON contributors.candidate_id = candidates.id
  WHERE
  candidates.last_name = "Huckabee"
  '''

display(viz_tables(['last_name', 'first_name', 'last_name'], query_2))


Inner join - part 1


,last_name,first_name
0,Huckabee,Steven
1,Huckabee,Don
2,Huckabee,Don
3,Huckabee,Don
4,Huckabee,Charles
...,...,...
170,McCain,MERRILL
171,McCain,GEORGE
172,McCain,PETER
173,McCain,SALEM


Inner join - part 2


,last_name,first_name
0,Huckabee,Steven
1,Huckabee,Don
2,Huckabee,Don
3,Huckabee,Don
4,Huckabee,Charles
5,Huckabee,Mike
6,Huckabee,Rebecca
7,Huckabee,Brittni
8,Huckabee,John D.
9,Huckabee,John D.


## `LEFT JOIN` or `LEFT OUTER JOIN`
There are many ways to combine two tables.  We just explored one possibility in which we combined the tables based upon the intersection of the two tables (the `INNER JOIN`).

Now we'll talk about the `LEFT JOIN` or `LEFT OUTER JOIN`.

In words, the `LEFT JOIN` is combining the tables based upon what is in the intersection of the two tables *and* what is in the "reference" table.

We can consider our toy example in two guises:

#### Example A
Let's do a `LEFT JOIN` of table B from table A.  That is, we'd like to make a new table by putting table B into table A.  In this case, we'll consider table A our "reference" table.  We're comparing by the `id` column again.  We know that these two tables share ids 0 and 2 and table A doesn't have anything else in it.  The resulting table is:

| nA   | A.attr  | nB  | B.attr |
| :::: | ::::::: | ::: | :::::: |
| s1   | 23      | t1  | 60     |
| s2   | 7       | t3  | 22     |

That's not very exciting.  It's the same result as from the `INNER JOIN`.  We can do another example that may be more enlightening.

#### Example B
Let's do a `LEFT JOIN` of table A from table B.  That is, we'd like to make a new table by putting table A into table B.  In this case, we'll consider table B our "reference" table.  Again, we use the `id` column from comparison.  We know that these two tables share ids 0 and 2.  This time, table B also contains the id 7, which is not shared by table A.  The resulting table is:

| nA   | A.attr  | nB  | B.attr |
| :::: | ::::::: | ::: | :::::: |
| s1   | 23      | t1  | 60     |
| None | NaN     | t2  | 14     |
| s2   | 7       | t3  | 22     |

Notice that `SQLite` filed in the missing entries for us.  This is necessary for completion of the requested join.

The `SQLite` commands to accomplish all of this are:

```SQL
SELECT nA, A.attr, nB, B.attr FROM A LEFT JOIN B ON B.idB = A.idA
```

and

```SQL
SELECT nA, A.attr, nB, B.attr FROM B LEFT JOIN A ON A.idA = B.idB
```

Here is a visualization using Venn diagrams of the `LEFT JOIN`.

![](../fig/left_join.png)

## Exercises
Use the following two tables to do the first two exercises in this section. Table A has the form:

| nA   | attr  | idA |
| :::: | ::::: | ::: |
| s1   | 23    | 0   |
| s2   | 7     | 2   |
| s3   | 15    | 3   |
| s4   | 31    | 7   |

and table B has the form:

| nB   | attr  | idB |
| :::: | ::::: | ::: |
| t1   | 60    | 0   |
| t2   | 14    | 7   |
| t3   | 22    | 2   |

1. Draw the table that would result from a `LEFT JOIN` using table A as the reference and the `id` columns for comparison.
2. Draw the table that would result from a `LEFT JOIN` using table B as the reference and the `id` columns for comparison.
3. Now back to the candidates and their contributors.  Create a new table with the following form:

| average contribution | candidate last name |
| :::::::::::::::::::: | ::::::::::::::::::: |
|    ...               |    ...              |

   The table should be created using the `LEFT JOIN` clause on the contributors table by joining the candidates table by the `id` column.  The `average contribution` column should be obtained using the `AVG` `SQL` function. Use the `GROUP BY` clause on the candidates last name.

In [38]:
# Creating a new table using the candidates and contributors.
query = '''SELECT
  candidates.last_name,
  AVG(contributors.amount) AS avg_contribution
FROM
  contributors
LEFT JOIN
  candidates
  ON contributors.candidate_id = candidates.id
GROUP BY
    candidates.last_name
  '''

display(viz_tables(['last_name', 'avg_contribution'], query))

,last_name,avg_contribution
0,Clinton,50.6000
1,Giuliani,-1360.0000
2,Huckabee,772.0000
3,McCain,452.6000
4,Obama,472.4356
5,Paul,-519.7088
6,Romney,156.0000


---

# `pandas`

We've been working with databases for the last few lectures and learning `SQLite` commands to work with and manipulate the databases.  There is a `Python` package called `pandas` that provides broad support for data structures.  It can be used to interact with relationsional databases through its own methods and even through `SQL` commands.

In the last part of this lecture, you will get to redo a bunch of the database exercises using `pandas`.

We won't be able to cover `pandas` from the ground up, but it's a well-documented library and is fairly easy to get up and running.  Here's the website:  [`pandas`](http://pandas.pydata.org/).

## Reading a datafile into `pandas`

In [39]:
# Using pandas naming convention
dfcand = pd.read_csv("candidates.txt", sep="|")
dfcand

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D
5,22,Rudolph,Giuliani,NaN,R
6,24,Mike,Gravel,NaN,D
7,16,Mike,Huckabee,NaN,R
8,30,Duncan,Hunter,NaN,R
9,31,Dennis,Kucinich,NaN,D


In [40]:
dfcontr = pd.read_csv("contributors.txt", sep="|")
dfcontr

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16
...,...,...,...,...,...,...,...,...,...,...,...,...
170,NaN,ABESHAUS,MERRILL,M.,1801 N. HEREFORD DRIVE,NaN,FLAGSTAFF,AZ,860011121,120.0,2008-01-16,37
171,NaN,ABRAHAM,GEORGE,NaN,P.O. BOX 1504,NaN,LAKE CHARLES,LA,706021504,800.0,2008-01-17,37
172,NaN,ABRAHAMSON,PETER,J.,1030 W. ROSCOE STREET,NaN,CHICAGO,IL,606572207,50.0,2008-01-25,37
173,NaN,ABRAHAM,SALEM,A.,P.O. BOX 7,NaN,CANADIAN,TX,790140007,1000.0,2008-01-17,37


Reading things in is quite easy with `pandas`.

Notice that `pandas` populates empty fields with `NaN` values.

The `id` column in the contributors dataset is superfluous.  Let's delete it.

In [41]:
del dfcontr['id']
dfcontr.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


Very nice!  And we used the `head` method to print out the first five rows.

## Creating a Table with `pandas`
We can use `pandas` to create tables in a database.

First, let's create a new database since we've already done a lot on our test database.

In [42]:
dbp = sqlite3.connect('L21_pandas_DB.sqlite')
csr = dbp.cursor()
csr.execute("DROP TABLE IF EXISTS candidates")
csr.execute("DROP TABLE IF EXISTS contributors")
csr.execute("PRAGMA foreign_keys=1")

csr.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_name TEXT, 
               party TEXT NOT NULL)''')

dbp.commit() # Commit changes to the database

csr.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

dbp.commit()

Last time, we opened the data files with `Python` and then manually used `SQLite` commands to populate the individual tables.  We can use `pandas` instead like so.

In [43]:
dfcand.to_sql("candidates", dbp, if_exists="append", index=False)

How big is our table?

In [44]:
dfcand.shape

(17, 5)

We can visualize the data in our `pandas`-populated table.  No surprises here except that `pandas` did everything for us.

In [45]:
query = '''SELECT * FROM candidates'''
csr.execute(query).fetchall()

[(16, 'Mike', 'Huckabee', None, 'R'),
 (20, 'Barack', 'Obama', None, 'D'),
 (22, 'Rudolph', 'Giuliani', None, 'R'),
 (24, 'Mike', 'Gravel', None, 'D'),
 (26, 'John', 'Edwards', None, 'D'),
 (29, 'Bill', 'Richardson', None, 'D'),
 (30, 'Duncan', 'Hunter', None, 'R'),
 (31, 'Dennis', 'Kucinich', None, 'D'),
 (32, 'Ron', 'Paul', None, 'R'),
 (33, 'Joseph', 'Biden', None, 'D'),
 (34, 'Hillary', 'Clinton', 'R.', 'D'),
 (35, 'Mitt', 'Romney', None, 'R'),
 (36, 'Samuel', 'Brownback', None, 'R'),
 (37, 'John', 'McCain', None, 'R'),
 (38, 'Tom', 'Tancredo', None, 'R'),
 (39, 'Christopher', 'Dodd', 'J.', 'D'),
 (41, 'Fred', 'Thompson', 'D.', 'R')]

## Querying a table with `pandas`

### One Way

In [46]:
dfcand.query("first_name=='Mike' & party=='D'")

,id,first_name,last_name,middle_name,party
6,24,Mike,Gravel,NaN,D


### Another Way

In [47]:
dfcand[(dfcand.first_name=="Mike") & (dfcand.party=="D")]

,id,first_name,last_name,middle_name,party
6,24,Mike,Gravel,NaN,D


### More Queries

In [48]:
dfcand[dfcand.middle_name.notnull()]

,id,first_name,last_name,middle_name,party
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
16,41,Fred,Thompson,D.,R


In [49]:
dfcand[dfcand.first_name.isin(['Mike', 'Hillary'])]

,id,first_name,last_name,middle_name,party
2,34,Hillary,Clinton,R.,D
6,24,Mike,Gravel,NaN,D
7,16,Mike,Huckabee,NaN,R


## Exercises
1. Use `pandas` to populate the contributors table.
2. Query the contributors tables with the following:
  1. List entries where the state is "VA" and the amount is less than $\$400.00$.
  2. List entries where the state is "NULL".
  3. List entries for the states of Texas and Pennsylvania.
  4. List entries where the amount contributed is between $\$10.00$ and $\$50.00$.

In [50]:
dbp = sqlite3.connect('L22_pandas_DB.sqlite')
csr = dbp.cursor()
csr.execute("DROP TABLE IF EXISTS contributors")
csr.execute("PRAGMA foreign_keys=1")

csr.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

dbp.commit()

In [51]:
#List entries where the state is "VA" and the amount is less than $400.00.
dfcontr.query("state=='VA' & amount<400")

#List entries where the state is "NULL".
dfcontr.query("state!=state")

#List entries for the states of Texas and Pennsylvania.
dfcontr.query("state=='TX' & state=='PA'" )

#List entries where the amount contributed is between $10.00 and $50.00.
dfcontr.query("amount>=10 & amount<=50")

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
18,Ardle,William,NaN,412 Dakota Avenue,NaN,Springfield,OH,45504,50.0,2007-06-28,16
25,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,50.0,2007-07-30,20
26,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,25.0,2007-08-16,20
34,Buck,Barbara,NaN,1780 NE 138th St,NaN,North Miami,FL,331811316,50.0,2007-09-13,20
35,Buck,Barbara,NaN,1780 NE 138th St,NaN,North Miami,FL,331811316,50.0,2007-07-19,20
38,Buchanek,Elizabeth,NaN,7917 Kentbury Dr,NaN,Bethesda,MD,208144615,50.0,2007-09-30,20
49,Harrison,Ryan,NaN,2247 3rd St,NaN,La Verne,CA,917504918,25.0,2007-07-26,20
101,Aarons,Elaine,NaN,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300,25.0,2008-02-26,34
104,Aaron,Shirley,NaN,101 Cherry Ave,NaN,Havana,FL,323331311,50.0,2008-02-29,34


## Sorting

In [52]:
dfcand.sort_values(by='party')

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D
13,29,Bill,Richardson,NaN,D
6,24,Mike,Gravel,NaN,D
11,20,Barack,Obama,NaN,D
9,31,Dennis,Kucinich,NaN,D
14,35,Mitt,Romney,NaN,R
12,32,Ron,Paul,NaN,R


In [53]:
dfcand.sort_values(by='party', ascending=False)

,id,first_name,last_name,middle_name,party
8,30,Duncan,Hunter,NaN,R
7,16,Mike,Huckabee,NaN,R
15,38,Tom,Tancredo,NaN,R
14,35,Mitt,Romney,NaN,R
12,32,Ron,Paul,NaN,R
10,37,John,McCain,NaN,R
1,36,Samuel,Brownback,NaN,R
16,41,Fred,Thompson,D.,R
5,22,Rudolph,Giuliani,NaN,R
6,24,Mike,Gravel,NaN,D


## Selecting Columns

In [54]:
dfcand[['last_name', 'party']]

,last_name,party
0,Biden,D
1,Brownback,R
2,Clinton,D
3,Dodd,D
4,Edwards,D
5,Giuliani,R
6,Gravel,D
7,Huckabee,R
8,Hunter,R
9,Kucinich,D


In [55]:
dfcand[['last_name', 'party']].count()

last_name    17
party        17
dtype: int64

In [56]:
dfcand[['first_name']].drop_duplicates()

,first_name
0,Joseph
1,Samuel
2,Hillary
3,Christopher
4,John
5,Rudolph
6,Mike
8,Duncan
9,Dennis
11,Barack


In [57]:
dfcand[['first_name']].drop_duplicates().count()

first_name    15
dtype: int64

## Exercises
1. Sort the contributors table by `amount` and order in *descending* order.
2. Select the `first_name` and `amount` columns.
3. Select the `last_name` and `first_name` columns and drop duplicates.
4. Count how many there are after the duplicates have been dropped.

In [58]:
#Sort the contributors table by amount and order in descending order.
print('Sort the contributors table by amount desc')
display(dfcontr.sort_values(by='amount', ascending=False))

#Select the first_name and amount columns.
print('Select the first_name and amount columns.')
display(dfcontr[['first_name', 'last_name']])

# Select the last_name and first_name columns and drop duplicates.
print('Select the last_name and first_name columns and drop duplicates.')
display(dfcontr[["last_name", "first_name"]].drop_duplicates())

# Count how many there are after the duplicates have been dropped.
print('Count how many there are after the duplicates have been dropped.')
display(dfcontr[["last_name", "first_name"]].drop_duplicates().count())

Sort the contributors table by amount desc


,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
30,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,4600.0,2007-08-14,20
159,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,NaN,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37
15,Anthony,John,NaN,211 Long Island Drive,NaN,Hot Springs,AR,71913,2300.0,2007-06-12,16
33,Buck,Blaine,M,45 Eaton Ave,NaN,Camden,ME,48431752,2300.0,2007-09-30,20
28,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,2300.0,2007-08-14,20
...,...,...,...,...,...,...,...,...,...,...,...
57,BURTON,STEVEN,G.,9938 DEER CREEK DRIVE,NaN,TAMPA,FL,33647,-2300.0,2008-03-05,22
111,Reich,Thomas,NaN,499 Park Ave,NaN,New York,NY,100221240,-2300.0,2008-08-28,34
58,BURTON,GLENN,M.,4404 CHARLESTON COURT,NaN,TAMPA,FL,336092620,-2300.0,2008-03-05,22
55,BUSH,ERIC,NaN,P.O. BOX 61046,NaN,DENVER,CO,802061046,-2300.0,2008-03-06,22


Select the first_name and amount columns.


,first_name,last_name
0,Steven,Agee
1,Don,Ahrens
2,Don,Ahrens
3,Don,Ahrens
4,Charles,Akin
...,...,...
170,MERRILL,ABESHAUS
171,GEORGE,ABRAHAM
172,PETER,ABRAHAMSON
173,SALEM,ABRAHAM


Select the last_name and first_name columns and drop duplicates.


,last_name,first_name
0,Agee,Steven
1,Ahrens,Don
4,Akin,Charles
5,Akin,Mike
6,Akin,Rebecca
...,...,...
169,ABERCROMBIE,DENIS
170,ABESHAUS,MERRILL
171,ABRAHAM,GEORGE
172,ABRAHAMSON,PETER


Count how many there are after the duplicates have been dropped.


last_name     126
first_name    126
dtype: int64

## Altering Tables

Creating a new column is quite easy with `pandas`.

In [59]:
dfcand['name'] = dfcand['last_name'] + ", " + dfcand['first_name']
dfcand

,id,first_name,last_name,middle_name,party,name
0,33,Joseph,Biden,NaN,D,"Biden, Joseph"
1,36,Samuel,Brownback,NaN,R,"Brownback, Samuel"
2,34,Hillary,Clinton,R.,D,"Clinton, Hillary"
3,39,Christopher,Dodd,J.,D,"Dodd, Christopher"
4,26,John,Edwards,NaN,D,"Edwards, John"
5,22,Rudolph,Giuliani,NaN,R,"Giuliani, Rudolph"
6,24,Mike,Gravel,NaN,D,"Gravel, Mike"
7,16,Mike,Huckabee,NaN,R,"Huckabee, Mike"
8,30,Duncan,Hunter,NaN,R,"Hunter, Duncan"
9,31,Dennis,Kucinich,NaN,D,"Kucinich, Dennis"


We can change an existing field as well.

In [60]:
dfcand.loc[dfcand.first_name == "Mike", "name"]

6      Gravel, Mike
7    Huckabee, Mike
Name: name, dtype: object

In [61]:
dfcand.loc[dfcand.first_name == "Mike", "name"] = "Mikey"

In [62]:
dfcand.query("first_name == 'Mike'")

,id,first_name,last_name,middle_name,party,name
6,24,Mike,Gravel,NaN,D,Mikey
7,16,Mike,Huckabee,NaN,R,Mikey


You may recall that `SQLite` doesn't have the functionality to drop a column.  It's a one-liner with `pandas`.

In [63]:
del dfcand['name']
dfcand

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D
5,22,Rudolph,Giuliani,NaN,R
6,24,Mike,Gravel,NaN,D
7,16,Mike,Huckabee,NaN,R
8,30,Duncan,Hunter,NaN,R
9,31,Dennis,Kucinich,NaN,D


## Exercises
1. Create a name column for the contributors table with field entries of the form "last name, first name"
2. For contributors from the state of "PA", change the name to "X".
3. Delete the newly created name column.

In [64]:
# Create a name column for the contributors table with field entries of the form "last name, first name"
dfcontr['name'] = dfcontr['last_name'] + ', ' + dfcontr['first_name']
display(dfcontr['name'])

# For contributors from the state of "PA", change the name to "X".
dfcontr.loc[dfcontr.state == "PA"] = "X"
display(dfcontr["state"])

# Delete the newly created name column.
del dfcontr['name']

0           Agee, Steven
1            Ahrens, Don
2            Ahrens, Don
3            Ahrens, Don
4          Akin, Charles
             ...        
170    ABESHAUS, MERRILL
171      ABRAHAM, GEORGE
172    ABRAHAMSON, PETER
173       ABRAHAM, SALEM
174       ABRAHAM, SALEM
Name: name, Length: 175, dtype: object

0      VA
1      CA
2      CA
3      CA
4      AR
       ..
170    AZ
171    LA
172    IL
173    TX
174    TX
Name: state, Length: 175, dtype: object

## Aggregation
We'd like to get information about the tables such as the maximum amount contributed to the candidates.  Here are a bunch of ways to describe the tables.

In [65]:
dfcand.describe()

,id
count,17.000000
mean,30.764706
std,7.110990
min,16.000000
25%,26.000000
50%,32.000000
75%,36.000000
max,41.000000


It's not very interesting with the candidates table because the candidates table only has one numeric column.

## Exercise
Use the `describe()` method on the `contributors` table.

In [66]:
dfcontr.describe()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
count,175,175,66,175,13,175,174,175,175.0,175,175
unique,94,102,21,118,7,105,35,118,50.0,86,8
top,Aaron,John,M.,2025 NW 29th Rd,X,Boca Raton,FL,334316303,100.0,2008-04-21,20
freq,16,12,10,9,5,9,30,9,25.0,21,25


In [67]:
db.commit()
db.close()

I'll use the contributors table to do some demos now.

In [68]:
dfcontr.amount.max()

TypeError: '>=' not supported between instances of 'float' and 'str'

In [ ]:
dfcontr[dfcontr.amount==dfcontr.amount.max()]

In [ ]:
dfcontr.groupby("state").sum()

In [ ]:
dfcontr.groupby("state")["amount"].sum()

In [ ]:
dfcontr.state.unique()

There is also a version of the `LIMIT` clause.  It's very intuitive with `pandas`.

In [ ]:
dfcand[0:3]

The usual `Python` slicing works just fine!

## Joins with `pandas`

`pandas` has some some documentation on `joins`:  [Merge, join, and concatenate](http://pandas.pydata.org/pandas-docs/stable/merging.html).  If you want some more reinforcement on the concepts from earlier regarding `JOIN`, then the `pandas` documentation may be a good place to get it.

You may also be interested in [a comparison with `SQL`](http://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html#compare-with-sql-join).

To do joins with `pandas`, we use the `merge` command.

Here's an example of an explicit inner join:

In [ ]:
cols_wanted = ['last_name_x', 'first_name_x', 'candidate_id', 'id', 'last_name_y']
dfcontr.merge(dfcand, left_on="candidate_id", right_on="id")[cols_wanted]

#### Somewhat organized example

In [ ]:
dfcontr.merge(dfcand, left_on="candidate_id", right_on="id")[cols_wanted].groupby('last_name_y').describe()

### Other Joins with `pandas`
We didn't cover all possible joins because `SQLite` can only handle the few that we did discuss.  As mentioned, there are workarounds for some things in `SQLite`, but not evertyhing.  Fortunately, `pandas` can handle pretty much everything.  Here are a few joins that `pandas` can handle:
* `LEFT OUTER` (already discussed)
* `RIGHT OUTER` - Think of the "opposite" of a `LEFT OUTER` join (shade the intersection and *right* set in the Venn diagram).
* `FULL OUTER` - Combine everything from both tables (shade the entire Venn diagram)

#### Left Outer Join with `pandas`

In [ ]:
dfcontr.merge(dfcand, left_on="candidate_id", right_on="id", how="left")[cols_wanted]

#### Right Outer Join with `pandas`

In [ ]:
dfcontr.merge(dfcand, left_on="candidate_id", right_on="id", how="right")[cols_wanted]

#### Full Outer Join with `pandas`

In [ ]:
dfcontr.merge(dfcand, left_on="candidate_id", right_on="id", how="outer")[cols_wanted]